# Eligibility Traces
$$
\newcommand{\td}[1]{\text{TD}(#1)}
\newcommand{\w}{\mathbf{w}}
\newcommand{\x}{\mathbf{x}}
\newcommand{\z}{\mathbf{z}}
\newcommand{\hv}[2]{\hat{v}(S_{#1}, \w_#2)}
$$
Eligibility traces are one of the most frequently used techniques in Reinforcement Learning.

Like $n$-step learning, they also form a continuum between Monte Carlo estimation and TD learning. Monte Carlo methods correspond to $\lambda = 1$ while TD learning corresponds to $\lambda = 0$.

Eligibility traces are implemented with a vector $\mathbf{z} \in \mathbb{R}^d$ with same dimensionality as the weight vector $\mathbf{w} \in \mathbb{R}^d$. This vector is known as the _eligibility trace_ and roughly corresponds to how recently a particular feature hsa been observed. The trace-decay parameter $\lambda$ determines how fast the $\mathbf{z}$ vector decays.

One advantage of eligibility traces over $n$-step learning is their computational complexity. Only a single eligibility trace vector needs to be stored as opposed to $n$ feature vectors.

Another advantage is that eligibility traces allow for immediate online learning, whereas $n$-step learning requires $n$ subsequent steps to be made before an update will be processed.

Eligibility traces provide an example of a _backwards view_ algorithm, where each new observed state is used to perform updates based on earlier state visitations, as opposed to the _forwards view_, where a state value is updated based on the future results acquired subsequent to that state.

## 12.1 The $\lambda$-return

We know that $n$-step TD updates targeting

$$
G_{t:t+n} = R_{t+1} + \gamma R_{t+2} + \dots \gamma^{n - 1}R_{t + n} + \gamma^n \hat{v}(S_{t+n}, \mathbf{w}_{t+n-1})
$$

eventually converge to true values. However, we note that any _average_ across $n$-step targets would also converge. For example, the update with target

$$
G_t = \frac{1}{2}G_{t:t+2} + \frac{1}{2}G_{t:t+4}
$$

should also converge.

As long as the weights on the $n$-step returns are positive and sum to $1$, any such convex combination is also a valid update target.

Averaging different updates is known as _compound updating_.

$\td{\lambda}$ can be thought of a compounding $n$-step updates for all values of $n$, where we weight each update by $(1 - \lambda)\lambda^n$.

The target for $\td{\lambda}$ can be defined written as

$$
G_t^\lambda = (1-\lambda)\sum_{n=1}^\infty{\lambda^{n-1}G_{t:t+n}}
$$

in the continuing case, and as

$$
G_t^\lambda = (1-\lambda)\sum_{n=1}^{T-t-1}{\lambda^{n-1}G_{t:t+n}} + \lambda^{T-t-1}G_t
$$

in the episodic case with $T$ the time of episode termination.

In the episodic case, note that when $\lambda = 1$, we have $G_t^\lambda = G_t$, which is the update for Monte Carlo, and when $\lambda = 0$, we have that $G_t^\lambda = G_{t:t+1}$, which is the update for $\td{0}$.

> _Exercise 12.1_ Just as the return be written recursively in terms of the first reward and itself one-step later (3.9), so can the $\lambda$-return. Derive the analogous recursive relationship from (12.2) and (12.1).

$$
\begin{align}
G_t^\lambda &= (1-\lambda)\sum_{n=1}^\infty{\lambda^{n-1}G_{t:t+n}}\\\\
&= (1-\lambda)\left(G_{t:t+1} + \sum_{n=2}^\infty{\lambda^{n-1}G_{t:t+n}}\right)\\\\
&= (1-\lambda)G_{t:t+1} + (1-\lambda)\sum_{n=1}^\infty{\lambda^{n}\left(R_{t+1} + \gamma G_{t+1:t+1+n}\right)}\\\\
&= (1-\lambda)G_{t:t+1} + (1-\lambda)\sum_{n=1}^\infty{\lambda^{n}R_{t+1}} + \gamma\lambda(1-\lambda)\sum_{n=1}^\infty{\lambda^{n-1}G_{t+1:t+n+1}}\\\\
&= (1-\lambda)G_{t:t+1} + \lambda R_{t+1} + \gamma \lambda G_{t+1}^\lambda\\\\
&= (1-\lambda)R_{t+1} + (1-\lambda)\gamma \hv{t+1}{t} + \lambda R_{t+1} + \gamma \lambda G_{t+1}^\lambda\\\\
&= R_{t+1} + \gamma\hv{t+1}{t} + \gamma\lambda\left(G_{t+1}^\lambda - \hv{t+1}{t}\right)\\\\
&= R_{t+1} + \gamma\left((1-\lambda)\hv{t+1}{t} + \lambda G_{t+1}^\lambda\right)\\\\
&= G_{t:t+1} + \gamma\lambda\left(G_{t+1}^\lambda - \hv{t+1}{t}\right)
\end{align}
$$

> _Exercise 12.2_ The parameter $\lambda$ characterizes how fast the exponential weighting in Figure 12.2 falls off, and thus how far into the future the $\lambda$-return algorithm looks in determining its update. But a rate factor such as $\lambda$ is sometimes an awkward way of characterizing the speed of the decay. For some purposes it is better to specify a time constant, or half-life. What is the equation relating $\lambda$ and the half-life, $\tau_\lambda$, the time by which the weighting sequence will have fallen to half of its initial value?

We define $\tau_\lambda$ such that $(1-\lambda)\lambda^{\tau_\lambda} = \frac{1}{2}(1-\lambda)$. Therefore we have

$$
\tau_\lambda = -\frac{\ln{2}}{\ln{\lambda}}
$$

For $n > \tau_\lambda + 1$ we would have $(1 - \lambda)\lambda^{n-1} < \frac{1}{2}(1-\lambda)$

## 12.2 $\td{\lambda}$

$$
\delta_t \quad \dot{=}\quad R_{t+1} + \gamma \hat{v}(S_{t+1}, \w_t) - \hat{v}(S_t, \w_t)
$$

> _Exercise 12.3_ Some insight into how $\td{\lambda}$ can closely approximate the off-line $\lambda$-return algorithm can be gained by seeing that the latter's error term (in brackets in (12.4)) can be written as the sum of TD errors (12.6) for a single fixed $\w$. Show this, following the pattern of (6.6), and using the recursive relationship for the $\lambda$-return you obtained in Exercise 12.1.

$$
\begin{align}
G_t^\lambda - \hat{v}(S_t, \w_t) &= G_{t:t+1} + \gamma\lambda\left(G_{t+1}^\lambda - \hv{t+1}{t}\right) - \hv{t}{t}\\\\
&= \delta_t + \gamma\lambda\left(G_{t+1}^\lambda - \hv{t+1}{t}\right)\\\\
&= \delta_t + \gamma\lambda\left(\delta_{t+1} + \gamma\lambda\left(G_{t+2}^\lambda - \hv{t+2}{t}\right)\right)\\\\
&= \delta_t + \gamma\lambda\delta_{t+1} + \left(\gamma\lambda\right)^2\delta_{t+2} + \dots\\
&= \sum_{n=0}^\infty{\left(\gamma\lambda\right)^n\delta_{t+n}}
\end{align}
$$

> Exercise 12.4 Use your result from the preceding exercise to show that, if the weight updates over an episode were computed on each step but not actually used to change the weights ($\w$ remained fixed), then the sum of $\td{\lambda}$'s weight updates would be the same as the sum of the off-line $\lambda$-return algorithm's updates.

First, note that

$$
\z_t = \sum_{n=0}^t{\left(\gamma\lambda\right)^n\nabla\hv{t-n}{{t-n}}}
$$

Now, we can see that the sum of the weight updates for $\td{\lambda}$ is

$$
\begin{align}
\alpha\sum_{t=0}^\infty{\delta_t\z_t} &= \alpha\sum_{t=0}^\infty{\delta_t\sum_{n=0}^t{\left(\gamma\lambda\right)^n\nabla\hv{t-n}{{t-n}}}}\\\\
&= \alpha\sum_{n\le t}{\delta_t\left(\gamma\lambda\right)^n\nabla\hv{t-n}{{t-n}}}\\\\
\end{align}
$$

And for off-line $\lambda$-return, the sum of weight updates would be

$$
\begin{align}
\alpha\sum_{t=0}^\infty{\left[G_t^\lambda - \hv{t}{t}\right]\nabla\hv{t}{t}} &= \alpha\sum_{t=0}^\infty{\left[\sum_{n=0}^\infty{\left(\gamma\lambda\right)^n\delta_{t+n}}\right]\nabla\hv{t}{t}}\\\\
&= \alpha\sum_{t=0}^\infty{\sum_{n=0}^\infty{\left(\gamma\lambda\right)^n\delta_{t+n}}\nabla\hv{t}{t}}\\\\
&= \alpha\sum_{n\le t'}^\infty{\left(\gamma\lambda\right)^n\delta_{t'}}\nabla\hv{t'-n}{{t'}}
\end{align}
$$

$$
\begin{align}
G_{t:h}^\lambda &\dot = (1-\lambda) \sum_{n=1}^{h-t-1} \lambda ^{n-1} G_{t:t+n} + \lambda^{h-t-1} G_{t:h}, \hspace{5mm} 0 \leq t < h \leq T \tag{12.9} \\
G_{t:k}^\lambda &= \hat v(S_t, \mathbf{w}_{t-1}) + \sum_{i=t}^{t+k-1} (\gamma \lambda)^{i-t} \delta_i ^\prime \tag{12.10}\\
\delta_t & \dot = R_{t+1} + \gamma \hat v(S_{t+1}, \mathbf{w}_t) - \hat v(S_t, \mathbf{w}_{t-1})
\end{align}
$$

Also note that from (12.9) we can conclude:
$G_{t+1:h}^\lambda = (1-\lambda) \sum_{n=1}^{h-t-2} \lambda ^{n-1} G_{t+1:t+1+n} + \lambda^{h-t-2} G_{t+1:h}$

$$
\begin{align}
G_{t:h}^\lambda &= (1-\lambda) \sum_{n=1}^{h-t-1} \lambda ^{n-1} (R_{t+1} + \gamma G_{t+1:t+n}) + \lambda^{h-t-1} G_{t:h}\\
 &= (1-\lambda) \left [ R_{t+1} \sum_{n=1}^{h-t-1} \lambda ^{n-1}  + \gamma \hat v(S_{t+1}, \mathbf{w}) + \gamma \sum_{n=2}^{h-t-1} \lambda ^{n-1} G_{t+1:t+n} \right ] + \lambda^{h-t-1} G_{t:h} \tag{separating sum}\\
 &= (1-\lambda) \left [ R_{t+1} \frac{\lambda^{h-t-1} - 1}{\lambda - 1}  + \gamma \hat v(S_{t+1}, \mathbf{w}) + \gamma \sum_{n=2}^{h-t-1} \lambda ^{n-1} G_{t+1:t+n} \right ] + \lambda^{h-t-1} G_{t:h} \tag{simplifying sum}\\
 &= R_{t+1}(1 - \lambda^{h-t-1}) + \gamma (1-\lambda) \left [ \hat v(S_{t+1}, \mathbf{w}) + \sum_{n=2}^{h-t-1} \lambda ^{n-1} G_{t+1:t+n} \right ] + \lambda^{h-t-1} G_{t:h}\\
\end{align}
$$

> _Exercise 12.6_ Modify the pseudocode for Sarsa($\lambda$) to use dutch traces (12.11) without the other features of a true online algorithm. Assume linear function approx. and binary features.

In [5]:
𝓐 = [-1, 0, 1]

3-element Vector{Int64}:
 -1
  0
  1

In [58]:
init() = (rand() * 0.2) - 0.6, 0

init (generic function with 1 method)

In [6]:
step(s, a) = clamp(s[1] + s[2], -1.2, 0.5), clamp(s[2] + 0.001 * a - 0.0025cos(3s[1]), -0.07, 0.07)

step (generic function with 1 method)

In [7]:
is_terminal(s) = s[1] < -1.2

is_terminal (generic function with 1 method)

In [95]:
function 𝓕(s,a)
    tiles_x = [ (l, l +0.2) for l in -1.2:0.2:0.5 ]
    tiles_xp = [ (l, l + 0.01) for l in -0.07:0.01:0.07 ]
    tiles = cat(tiles_x, tiles_xp, dims=1)
    x = [1]
    for (i, t) in enumerate(tiles_x)
        if t[1] ≤ s[1] ≤ t[2]
            append!(x, i)
        end
    end
    
    for (i, t) in enumerate(tiles_xp)
        if t[1] ≤ s[2] ≤ t[2]
            append!(x, i + length(tiles_x))
        end
    end
    
    if a == -1
        append!(x, x[end] + 1)
    elseif a == 0
        append!(x, x[end] + 2)
    else
        append!(x, x[end] + 3)
    end
end

𝓕 (generic function with 1 method)

In [82]:
𝓕(init(), 1)

5-element Vector{Int64}:
  1
  4
 16
 17
 20

In [96]:
function sarsa_λ(is_terminal, γ, 𝓐, d, init, step, 𝓕; α=0.01, λ=0.5, ε=0.0001)
    w = zeros(d)
    q(s,a) = sum(w[i] for i in 𝓕(s, a))
    function x_(s,a)
        xp = zeros(d)
        for i in 𝓕(s, a)
            xp[i] = 1
        end
        return xp
    end
    z = zeros(d)
    s = init()
    a = argmax(a -> q(s, a), 𝓐)
    x = x_(s, a)

    while true
        r, sp = step(s, a)
        δ = r
        c = z' * x
        for i in 𝓕(s,a)
            δ -= w[i]
            z[i] += (1 - α*γ*λ*c)*x[i]
        end
        
        if is_terminal(sp)
            w += α*δ * z
            s = init()
            a = argmax(a -> q(s, a), 𝓐)
            continue
        end
        
        ap = argmax(a -> q(s,a), 𝓐)
        for i in 𝓕(s, a)
            δ += γ * w[i]
        end
        
        w += α * δ * z
        z *= λ * γ
        s = sp
        x = x_(s, a)
        a =ap
    end
end

sarsa_λ (generic function with 2 methods)

In [98]:
sarsa_λ(is_terminal, 0.99, 𝓐, 20, init, step, 𝓕)

LoadError: BoundsError

> _Exercise 12.7_ Generalize the three recursive equations above to their truncated versions, defining $G^{\lambda s}_{t:h}$ and $G^{\lambda a}_{t:h}$

$$
\begin{align}
G ^{\lambda s} _{t:h} &= 
    R_{t+1} + \gamma_{t + 1}
    \left(
    (1 - \lambda_t)\hat{v}(S_{t+1}, \w_t) + \lambda_{t+1}G^{\lambda s}_{t+1:h}
    \right)\\\\
G^{\lambda s}_{h:h} &= \left(1 - \sum_{n=1}^h\prod_{i=1}^n{\lambda_i}\right)G_{t:h}\\\\
G ^{\lambda a} _{t:h} &= 
    R_{t+1} + \gamma_{t + 1}
    \left(
    (1 - \lambda_t)\hat{q}(S_{t+1}, A_{t+1}\w_t) + \lambda_{t+1}G^{\lambda s}_{t+1:h}
    \right)\\\\
G^{\lambda a}_{h:h} &= \left(1 - \sum_{n=1}^h\prod_{i=1}^n{\lambda_i}\right)G_{t:h}\\\\
G' ^{\lambda a} _{t:h} &= 
    R_{t+1} + \gamma_{t + 1}
    \left(
    (1 - \lambda_t)\bar{V}(S_{t+1}, \w_t) + \lambda_{t+1}G^{\lambda s}_{t+1:h}
    \right)\\\\
G'^{\lambda a}_{h:h} &= \left(1 - \sum_{n=1}^h\prod_{i=1}^n{\lambda_i}\right)G_{t:h}
\end{align}
$$

with $\bar{V}$ as defined in the text.

> _Exercise 12.8_ Prove that (12.24) becomes exact if the value function does not change. To save writing, consider the case of $t = 0$, and use the notation $V_k \;\dot{=}\; \hat{v}(S_k, \w)$.

$$
\begin{align}
G_t^{\lambda s} &= \rho_t\left(R_{t+1} + \gamma_{t+1}\left((1-\lambda_{t+1})V_{t+1} + \lambda_{t+1}G^{\lambda s}_{t+1}\right)\right) + (1 - \rho_t)V_t\\\\
&= \rho_t R_{t+1} + \rho_t \gamma_{t+1}(1-\lambda_{t+1})V_{t+1} + \rho_t\gamma_{t+1}\lambda_{t+1}G^{\lambda s}_{t+1} + V_t - \rho_t V_t\\\\
&= \rho_t R_{t+1} + \rho_t \gamma_{t+1}V_{t+1} - \rho_t \gamma_{t+1}\lambda_{t+1}V_{t+1} + \rho_t\gamma_{t+1}\lambda_{t+1}G^{\lambda s}_{t+1} + V_t - \rho_t V_t\\\\
G_t - V_t&= \rho_t\delta_t - \rho_t\gamma_{t+1}\lambda_{t+1}V_{t+1} + \rho_t\gamma_{t+1}\lambda_{t+1}G_{t+1}\\\\
&= \rho_t\left(\delta_t + \gamma_{t+1}\lambda_{t+1}\left(G_{t+1} - V_{t+1}\right)\right)\\\\
&= \rho_t\left(\delta_t + \gamma_{t+1}\lambda_{t+1}\left(\rho_{t+1}\left(\delta_{t+1} + \gamma_{t+2}\lambda_{t+2}\left(G_{t+2} - V_{t+2}\right)\right)\right)\right)\\\\
&= \rho_t\left(\sum_{k=t}^\infty{\delta_k\prod_{i=t+1}^k{\gamma_i\lambda_i\rho_i}}\right)\\\\
G_t &= V_t + \rho_t\left(\sum_{k=t}^\infty{\delta_k\prod_{i=t+1}^k{\gamma_i\lambda_i\rho_i}}\right)\\\\
\end{align}
$$

$$
\delta_t = R_{t+1} + \gamma_{t+1}V_{t+1} - V_t
$$

> _Exercise 12.9_ The truncated version of the general off-policy return is denoted $G^{\lambda s}_{t:h}$.
Guess the correct equation, based on (12.24).

$$
G_t = V_t + \rho_t\left(\sum_{k=t}^{t+h}{\delta_k\prod_{i=t+1}^k{\gamma_i\lambda_i\rho_i}}\right)
$$

> _Exercise 12.10_ Prove that (12.27) becomes exact if the value function does not change. To save writing, consider the case of $t = 0$, and use the notation $Q_k = \hat{q}(S_k, A_k, \w)$. Hint: Start by writing out $\delta_0^a$ and $G_0^{\lambda a}$, then $G_0^{\lambda a} - Q_0$

$$
\begin{align}
%
    \delta_0 &= R_1 + \gamma_1\bar{V}(S_1) - Q_0\\\\
%
    G_0 &= R_1 + \gamma_1\left(\bar{V} + \lambda_{t+1}\rho_{t+1}\left[G_1 - Q_1\right]\right)\\\\
%
%
    G_0 - Q_0 &= R_1 + \gamma_1\left(\bar{V} + \lambda_{t+1}\rho_{t+1}\left[G_1 - Q_1\right]\right) - Q_0\\\\
    &= R_1 + \gamma_1\bar{V} - Q_0 + \gamma_1 \lambda_{t+1} \rho_{t+1}\left[G_1 - Q_1\right]\\\\
    &= \delta_0 + \gamma_1 \lambda_{1} \rho_{1}\left[G_1 - Q_1\right]\\\\
    &= \delta_0 + \gamma_1 \lambda_{1} \rho_{1}\left(\delta_1 + \gamma_2\lambda_2\left(G_2 - Q_2\right)\right)\\\\
    &= \sum_{k=0}^{\infty}{\delta_k\left(\prod_{i=0}^k{\gamma_i\lambda_i}\rho_i\right)}\\\\
\end{align}
$$

> _Exercise 12.11_ The truncated version of the general off-policy return is denoted $G^{\lambda a}_{t:h}$ Guess the correct equation for it, based on (12.27).

$$
G_t^{\lambda a} \approx \hat{q}(S_t, A_t, \mathbf{w}_t) + \sum_{k=t}^{t+h}{\delta_k^a \prod_{i=t+1}^k{\gamma_i\lambda_i\rho_i}}
$$

> Exercise 12.12 Show in detail the steps outlined above for deriving (12.29) from (12.27). Start with the update (12.15), substitute $G_t^{\lambda a}$ from (12.26) for $G_t^\lambda$, then follow similar steps as led to (12.25).

From 12.27 we have

$$
\begin{align*}
\sum_{t=0}^\infty{\left(\w_{t+1} - \w_t\right)} &= \alpha \sum_{t=0}^\infty{\sum_{k=t}^\infty{\rho_t \delta_k^a \prod_{i=t+1}^k{\gamma_i \lambda_i \rho_i}}\nabla Q_t}\\\\
% 
&= \alpha \sum_{k=0}^\infty{\sum_{t=0}^k{\rho_t \delta_k^a \prod_{i=t+1}^k{\gamma_i \lambda_i \rho_i}}\nabla Q_t}\\\\
% 
&= \sum_{k=0}^\infty{\alpha \delta_k \sum_{t=0}^k{\rho_t \nabla Q_t\prod_{i=t+1}^k{\gamma_i \lambda_i \rho_i}}}
\end{align*}
$$

Where $\nabla Q_t = \nabla \hat{q}(S_t, A_t, \w_t)$.

Following the earlier derivation, we assume that the inner sum can be written recursively, as:

$$
\begin{align*}
\z_k &= \sum_{t=0}^k{\rho_t \nabla Q_t\prod_{i=t+1}^k{\gamma_i \lambda_i \rho_i}}\\\\
&= \sum_{t=0}^{k-1}{\rho_t \nabla Q_t\prod_{i=t+1}^k{\gamma_i \lambda_i \rho_i}} + \rho_k \nabla Q_k\\\\
% 
&= \gamma_k \lambda_k \rho_k \left(\sum_{t=0}^{k-1}{\rho_t \nabla Q_t\prod_{i=t+1}^{k-1}{\gamma_i \lambda_i \rho_i}}\right) + \rho_k \nabla Q_k\\\\
% 
&= \gamma_k \lambda_k \rho_k \z_{k-1} + \rho_k \nabla Q_k\\\\
% 
&= \rho_k \left( \gamma_k \lambda_k \z_{k-1} + \nabla Q_k \right)
\end{align*}
$$



> Exercise 12.13 What are the dutch-trace and replacing-trace versions of off-policy eligibility traces for state-value and action-value methods?